In [1]:
pip install nltk pandas sacremoses transformers sentencepiece datasets torch evaluate

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install sentence_transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch transformers pandas tqdm

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd

In [9]:
df.to_csv("aligned_merged_sentences.csv", index=False, encoding="utf-8")

In [7]:
print(df.shape)
print(df.head(10))

(50000, 2)
                                             Kannada  \
0                             ಚುನಾವಣೆ ನಡೆಯುವುದಿಲ್ಲ .   
1  ಮಾತು ಮುಂದುವರೆಸಿದ ಮಾಜಿ ಸಚಿವ ವಿ. ಶ್ರೀನಿವಾಸ್ ಪ್ರಸ...   
2  ತನ್ನ ಪರೀಕ್ಷಾ ಸಾಮರ್ಥ್ಯವನ್ನು ಹೆಚ್ಚಿಸಿಕೊಳ್ಳಲು, ವೈ...   
3                 ಹತ್ಯೆಗೆ ಸ್ಪಷ್ಟ ಕಾರಣ ತಿಳಿದುಬಂದಿಲ್ಲ.   
4     ಮತ್ತು ಇದನ್ನು ಬೆಳೆಸುವ ವಿಧಾನವೂ ಅಷ್ಟೇನು ಕಷ್ಟವಲ್ಲ.   
5  ಅಪೊಸ್ತಲ ಪೌಲನು ನಂಬಿಗಸ್ತಿಕೆಯಿಂದ ಸಾರುವ ಕಾರ್ಯದಲ್ಲಿ...   
6  18 ವರ್ಷದೊಳಗಿನ ಪತ್ನಿ ಜೊತೆ ಸಂಬಂಧ ಬೆಳೆಸಿದ್ರೆ ಅದನ್...   
7  ಮುಂಬೈ: ಪುಲ್ವಾಮಾ ದಾಳಿ ಬಳಿಕ ಭಾರತದಲ್ಲಿ ಪಾಕಿಸ್ತಾನ ...   
8              ಹೀಗೆ ಹಲವಾರು ಕಾರ್ಯಕ್ರಮಗಳನ್ನು ರೂಪಿಸಿದೆ.   
9                         ನೋಕಿಯಾ ಲುಮಿಯಾ 800 ವಿಶೇಷತೆ:   

                                             English  
0                              There is no election.  
1  Former Minister V. Sreenivasa Prasad will pres...  
2  To boost its testing capacity, Council of Scie...  
3        No specific reason for the murder is known.  
4      And it's not too difficult to do that either.  
5  The apostle Paul was physically beaten 

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine

similarity_model = SentenceTransformer('l3cube-pune/indic-sentence-similarity-sbert')
 
assert "Kannada" in df.columns and "English" in df.columns, "Missing Kannada or English columns in dataset!"

def calculate_cosine_similarity(text1, text2):
    embeddings1 = similarity_model.encode([text1])
    embeddings2 = similarity_model.encode([text2])
    similarity_score = 1 - cosine(embeddings1[0], embeddings2[0]) 
    return similarity_score

valid_indices = []

for idx, row in df.iterrows():
    similarity = calculate_cosine_similarity(row['Kannada'], row['English'])
    
    print(f"Comparing Kannada: '{row['Kannada']}'")
    print(f"With English: '{row['English']}'")
    print(f"Cosine Similarity: {similarity:.4f}")
    
    if similarity > 0.7:
        print(f"Sentence pair kept (cosine similarity > 0.7)")
        valid_indices.append(idx)
    else:
        print(f"Sentence pair removed (cosine similarity <= 0.7)")
    
    print('-' * 50)  

df_filtered = df.loc[valid_indices]

df_filtered.to_csv("filtered_dataset.csv", index=False)

print("Filtered Dataset (First 10 rows):")
print(df_filtered.head(10))

print("Filtering complete! Saved as 'filtered_dataset.csv'.")


In [7]:
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine

similarity_model = SentenceTransformer('l3cube-pune/indic-sentence-similarity-sbert')

model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Using device: cuda


Translating to mal_Mlym: 100%|██████████| 73/73 [00:12<00:00,  5.66it/s]

                                              Kannada  \
0                              ಚುನಾವಣೆ ನಡೆಯುವುದಿಲ್ಲ .   
2   ತನ್ನ ಪರೀಕ್ಷಾ ಸಾಮರ್ಥ್ಯವನ್ನು ಹೆಚ್ಚಿಸಿಕೊಳ್ಳಲು, ವೈ...   
3                  ಹತ್ಯೆಗೆ ಸ್ಪಷ್ಟ ಕಾರಣ ತಿಳಿದುಬಂದಿಲ್ಲ.   
4      ಮತ್ತು ಇದನ್ನು ಬೆಳೆಸುವ ವಿಧಾನವೂ ಅಷ್ಟೇನು ಕಷ್ಟವಲ್ಲ.   
6   18 ವರ್ಷದೊಳಗಿನ ಪತ್ನಿ ಜೊತೆ ಸಂಬಂಧ ಬೆಳೆಸಿದ್ರೆ ಅದನ್...   
8               ಹೀಗೆ ಹಲವಾರು ಕಾರ್ಯಕ್ರಮಗಳನ್ನು ರೂಪಿಸಿದೆ.   
9                          ನೋಕಿಯಾ ಲುಮಿಯಾ 800 ವಿಶೇಷತೆ:   
10  ಬದಲಾಗಿ ಯೋನಾತಾನನು ದಾವೀದನ ಆಪ್ತಮಿತ್ರನೂ ಸಹಾಯಕನೂ ಆಗ...   
11  ‘ ಒಂದು ಹಾಡಿನಲ್ಲಿ ದೇವರನ್ನು ಮಹಿಮೆಪಡಿಸುವುದು ಯಾವ ಅ...   
12                              ಅಮ್ಮು ನಿನಗೆ ನೆನಪಿದೆಯ?   

                                              English  \
0                               There is no election.   
2   To boost its testing capacity, Council of Scie...   
3         No specific reason for the murder is known.   
4       And it's not too difficult to do that either.   
6   Having sexual relationship with a girl below t...   
8               A number of ev

In [6]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset, DatasetDict
from peft import LoraConfig, get_peft_model

df = pd.read_csv("final_dataset.csv")  
df=df.head(100)
df = df[["Kannada", "Tamil", "Telugu", "Malayalam"]].dropna()

hf_dataset = Dataset.from_pandas(df)

dataset = hf_dataset.train_test_split(test_size=0.1)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Kannada', 'Tamil', 'Telugu', 'Malayalam'],
        num_rows: 90
    })
    test: Dataset({
        features: ['Kannada', 'Tamil', 'Telugu', 'Malayalam'],
        num_rows: 10
    })
})


In [7]:
model_name = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.to("cuda" if torch.cuda.is_available() else "cpu")


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [1]:
pip install transformers datasets torch sentencepiece matplotlib


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install transformers datasets sentencepiece torch accelerate sacrebleu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 152.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install transformers datasets torch sentencepiece accelerate evaluate sacrebleu


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, MT5ForConditionalGeneration, TrainingArguments, Trainer

df = pd.read_csv("final_dataset.csv")

data_pairs = []
for _, row in df.iterrows():
    for src_lang, tgt_lang in [("Kannada", "Tamil"), ("Kannada", "Telugu"), ("Kannada", "Malayalam")]:
        data_pairs.append({
            "input": f"<extra_id_0> {src_lang} → {tgt_lang}: {row[src_lang]}",
            "target": row[tgt_lang]
        })
        data_pairs.append({
            "input": f"<extra_id_0> {tgt_lang} → {src_lang}: {row[tgt_lang]}",
            "target": row[src_lang]
        })

dataset = Dataset.from_pandas(pd.DataFrame(data_pairs))

tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")

def tokenize_function(examples):
    model_inputs = tokenizer(examples["input"], max_length=128, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["target"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = dataset.shuffle(seed=42).select([i for i in range(45000)]) 
val_dataset = dataset.select([i for i in range(45000, 48618)])  

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir="./results",             
    evaluation_strategy="epoch",       
    learning_rate=2e-5,               
    per_device_train_batch_size=4,      
    per_device_eval_batch_size=4,      
    num_train_epochs=5,               
    weight_decay=0.01,                 
    logging_dir="./logs",              
    logging_steps=10,                   
    save_strategy="epoch",          
)

trainer = Trainer(
    model=model,                        
    args=training_args,                 
    train_dataset=train_dataset,        
    eval_dataset=val_dataset,           
    tokenizer=tokenizer,                
)

trainer.train()


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3618 [00:00<?, ? examples/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_41432/48624153.py:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.607400,0.446499
2,0.605200,0.407977
3,0.535200,0.394828
4,0.461200,0.388340
5,0.558100,0.386353


TrainOutput(global_step=56250, training_loss=1.2188820366753472, metrics={'train_runtime': 18111.8458, 'train_samples_per_second': 12.423, 'train_steps_per_second': 3.106, 'total_flos': 2.9742170112e+16, 'train_loss': 1.2188820366753472, 'epoch': 5.0})

In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


checkpoint_path = "results/checkpoint-56250"


model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)


save_path = "Sujal-mT5"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model and tokenizer saved to {save_path}")


Model and tokenizer saved to Sujal-mT5


In [1]:
from transformers import MT5ForConditionalGeneration, AutoTokenizer


model_path = "Sujal-mT5" 
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = MT5ForConditionalGeneration.from_pretrained(model_path)


In [3]:
def translate_text(input_text, target_language):
   
    input_text = f"<extra_id_0> Kannada → {target_language}: {input_text}"
    
    
    inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True, padding="max_length")
    
    
    output_ids = model.generate(inputs["input_ids"], num_beams=10, max_length=128, early_stopping=True)
    
  
    translated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    
    return translated_text


In [6]:
input_sentence = "ನಾನು ಶಾಲೆಗೆ ಹೋಗಿದ್ದೇನೆ"  

 
translated_sentence_tamil = translate_text(input_sentence, target_language="Tamil")
print("Translated to Tamil:", translated_sentence_tamil)


translated_sentence_telugu = translate_text(input_sentence, target_language="Telugu")
print("Translated to Telugu:", translated_sentence_telugu)

translated_sentence_malayalam = translate_text(input_sentence, target_language="Malayalam")
print("Translated to Malayalam:", translated_sentence_malayalam)


Translated to Tamil: நான் பள்ளியில் இருந்தேன்.
Translated to Telugu: నేను ಶಾಲೆಗೆ వచ్చాను
Translated to Malayalam: ഞാന് പഠിച്ചു.


In [ ]:
input_sentence = "నేను తెలుగు నేర్చుకుంటున్నాను"  

translated_sentence_telugu = translate_text(input_sentence, target_language="Kannada")
print("Translated to kannada:", translated_sentence_telugu)


Translated to kannada: ನಾನು ಕನ್ನಡದ ಬಗ್ಗೆ ಮಾತನಾಡಿದ್ದೇನೆ


In [5]:
input_sentence = "నేను ಶಾಲೆಗೆ వచ్చాను"  




translated_sentence_telugu = translate_text(input_sentence, target_language="Kannada")
print("Translated to kannada:", translated_sentence_telugu)


Translated to kannada: ನಾನು ಶಾಲೆಗೆ ಬಂದಿದ್ದೇನೆ


In [7]:
input_sentence = "ಅವಳು ನನಗೆ ಧನ್ಯವಾದಗಳು ಹೇಳಿದಳು."  

translated_sentence_tamil = translate_text(input_sentence, target_language="Tamil")
print("Translated to Tamil:", translated_sentence_tamil)


translated_sentence_telugu = translate_text(input_sentence, target_language="Telugu")
print("Translated to Telugu:", translated_sentence_telugu)


translated_sentence_malayalam = translate_text(input_sentence, target_language="Malayalam")
print("Translated to Malayalam:", translated_sentence_malayalam)


Translated to Tamil: அவருக்கு நன்றிகள்.
Translated to Telugu: అతను నాకు కృతజ్ఞతలు చెప్పారు.
Translated to Malayalam: അദ്ദേഹം എനിക്ക് നന്ദി പറഞ്ഞു.


In [ ]:
input_sentence = "ನಾನು ನಿಮ್ಮೊಂದಿಗೆ ಪ್ರಯಾಣ ಮಾಡುತ್ತಿರುವಾಗ ನಾನು ಹಲವು ಸುಂದರವಾದ ದೃಶ್ಯಗಳನ್ನು ನೋಡಿದೆನು." 


translated_sentence_tamil = translate_text(input_sentence, target_language="Tamil")
print("Translated to Tamil:", translated_sentence_tamil)


translated_sentence_telugu = translate _text(input_sentence, target_language="Telugu")
print("Translated to Telugu:", translated_sentence_telugu)


translated_sentence_malayalam = translate_text(input_sentence, target_language="Malayalam")
print("Translated to Malayalam:", translated_sentence_malayalam)


Translated to Tamil: நான் உங்கள் பயணத்தில் ಹಲವು அழகான காட்சிகளை பார்த்தேன்.
Translated to Telugu: నేను మీరు ప్రయాణించినప్పుడు చాలా అందమైన ప్రదేశాలు చూసాను.
Translated to Malayalam: എന്റെ കൂടെ യാത്ര ചെയ്യുമ്പോൾ ഞാന് വളരെ സന്തോഷിച്ചു.


In [2]:
from transformers import AutoModelForSeq2SeqLM
import torch

# Load your model from the local directory
model = AutoModelForSeq2SeqLM.from_pretrained("Sujal-mT5")

# Save its weights to a .pth file
torch.save(model.state_dict(), "sujal_mt5_model.pth")

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load model and tokenizer
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)



In [2]:

# Mapping from language name to NLLB-200 language code
LANG_CODE_MAP = {
    "Kannada": "kan_Knda",
    "Telugu": "tel_Telu",
    "Tamil": "tam_Taml",
    "Malayalam": "mal_Mlym"
}

def translate_from_input_format(input_line):
    """
    Translates a line in the format: <SourceLanguage> <Sentence> <TargetLanguage>
    
    Args:
        input_line (str): Full input string
    
    Returns:
        str: Translated sentence
    """
    words = input_line.strip().split()
    src_lang = words[0]
    tgt_lang = words[-1]
    sentence = " ".join(words[1:-1])
    
    src_code = LANG_CODE_MAP.get(src_lang)
    tgt_code = LANG_CODE_MAP.get(tgt_lang)

    if not src_code or not tgt_code:
        return f"Unsupported language(s): {src_lang}, {tgt_lang}"
    
    tokenizer.src_lang = src_code
    inputs = tokenizer(sentence, return_tensors="pt")

    translated_tokens = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_code),
        max_length=50
    )

    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    return translated_text


In [4]:
input_line = "Kannada ನಾನು ಆಟ ಆಡಲು ಹೋದೆ. Telugu"
translated = translate_from_input_format(input_line)
print("Translated Sentence:", translated)


Translated Sentence: నేను ఒక ఆట ఆడటానికి వెళ్ళాను.


In [3]:
input_line = "Telugu నేను ఒక ఆట ఆడటానికి వెళ్ళాను. Kannada"
translated = translate_from_input_format(input_line)
print("Translated Sentence:", translated)

Translated Sentence: ನಾನು ಆಟವಾಡಲು ಹೋಗಿದ್ದೆ.


In [2]:
pip install tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
from transformers import MT5ForConditionalGeneration, AutoTokenizer
import torch
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load your trained model/tokenizer
model_path = "Sujal-mT5"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = MT5ForConditionalGeneration.from_pretrained(model_path).to(device)
model.eval()

2025-05-15 09:31:47.704579: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-15 09:31:48.121733: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747301508.275247    1794 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747301508.330125    1794 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747301508.677053    1794 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [2]:
langs = ['Kannada', 'Tamil', 'Telugu', 'Malayalam']
lang_code_map = {
    'Kannada': 'Kannada',
    'Tamil': 'Tamil',
    'Telugu': 'Telugu',
    'Malayalam': 'Malayalam'
}


In [ ]:
langs = ['Kannada', 'Tamil', 'Telugu', 'Malayalam']
lang_code_map = {
    'Kannada': 'Kannada',
    'Tamil': 'Tamil',
    'Telugu': 'Telugu',
    'Malayalam': 'Malayalam'
}


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/950M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
def translate_batch(texts, src_lang, tgt_lang):
    prefix = f"translate {lang_code_map[src_lang]} to {lang_code_map[tgt_lang]}: "
    inputs = tokenizer([prefix + text for text in texts], return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=128)
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return decoded

def sentence_bleu_score(reference, candidate):
    return sentence_bleu(
        [reference.split()],
        candidate.split(),
        weights=(0.25, 0.25, 0.25, 0.25),
        smoothing_function=smoother
    )

def sentence_similarity(ref, cand):
    emb_ref = sim_model.encode(ref, convert_to_tensor=True)
    emb_cand = sim_model.encode(cand, convert_to_tensor=True)
    return util.pytorch_cos_sim(emb_ref, emb_cand).item()

In [5]:
import numpy as np
from tqdm import tqdm  # for progress bars

bleu_df = pd.DataFrame(index=langs, columns=langs, dtype=float)
sim_df = pd.DataFrame(index=langs, columns=langs, dtype=float)

# Compute BLEU and similarity for each language pair
for src_lang in langs:
    src_texts = df[src_lang].tolist()
    for tgt_lang in langs:
        tgt_texts = df[tgt_lang].tolist()
        if src_lang == tgt_lang:
            bleu_df.loc[src_lang, tgt_lang] = 100.0
            sim_df.loc[src_lang, tgt_lang] = 1.0
            print(f"Skipping {src_lang} -> {tgt_lang} (same language), BLEU=100, Similarity=1.0")
            continue

        print(f"\nTranslating {src_lang} -> {tgt_lang} ...")
        translations = translate_batch(src_texts, src_lang, tgt_lang)

        bleu_scores = []
        sim_scores = []
        # Use tqdm for sentence-level progress bar
        for i, (ref, pred) in enumerate(tqdm(zip(tgt_texts, translations), total=len(tgt_texts), desc=f"{src_lang}->{tgt_lang}")):
            try:
                bleu_val = sentence_bleu_score(ref, pred)
            except Exception as e:
                bleu_val = 0.0
            sim_val = sentence_similarity(ref, pred)
            bleu_scores.append(bleu_val)
            sim_scores.append(sim_val)

            # Print progress every 100 sentences (optional)
            if (i + 1) % 100 == 0 or i == len(tgt_texts) - 1:
                avg_bleu = np.mean(bleu_scores) * 100
                avg_sim = np.mean(sim_scores)
                print(f"  [{i+1}/{len(tgt_texts)}] Avg BLEU: {avg_bleu:.2f}%, Avg Similarity: {avg_sim:.4f}")

        avg_bleu = np.mean(bleu_scores) * 100
        avg_sim = np.mean(sim_scores)

        bleu_df.loc[src_lang, tgt_lang] = avg_bleu
        sim_df.loc[src_lang, tgt_lang] = avg_sim

print("\nBLEU Scores Matrix (%):")
print(bleu_df.round(2))

print("\nSimilarity Scores Matrix:")
print(sim_df.round(4))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Skipping Kannada -> Kannada (same language), BLEU=100, Similarity=1.0

Translating Kannada -> Tamil ...


Kannada->Tamil: 100%|██████████| 100/100 [00:02<00:00, 38.28it/s]


  [100/100] Avg BLEU: 5.52%, Avg Similarity: 0.4903

Translating Kannada -> Telugu ...


Kannada->Telugu: 100%|██████████| 100/100 [00:02<00:00, 41.05it/s]


...

  [100/100] Avg BLEU: 5.07%, Avg Similarity: 0.4510

Translating Malayalam -> Tamil ...


Malayalam->Tamil: 100%|██████████| 100/100 [00:01<00:00, 53.39it/s]


  [100/100] Avg BLEU: 4.96%, Avg Similarity: 0.4537

Translating Malayalam -> Telugu ...


Malayalam->Telugu: 100%|██████████| 100/100 [00:01<00:00, 53.54it/s]

  [100/100] Avg BLEU: 2.69%, Avg Similarity: 0.4958
Skipping Malayalam -> Malayalam (same language), BLEU=100, Similarity=1.0

BLEU Scores Matrix (%):
           Kannada   Tamil  Telugu  Malayalam
Kannada     100.00    5.52    6.75       5.39
Tamil         4.94  100.00    5.98       3.22
Telugu        5.01    5.08  100.00       3.25
Malayalam     5.07    4.96    2.69     100.00

Similarity Scores Matrix:
           Kannada   Tamil  Telugu  Malayalam
Kannada     1.0000  0.4903  0.4484     0.5168
Tamil       0.5013  1.0000  0.5146     0.5163
Telugu      0.4613  0.4905  1.0000     0.5143
Malayalam   0.4510  0.4537  0.4958     1.0000


In [19]:
import pandas as pd
import numpy as np
import torch
from transformers import MT5ForConditionalGeneration, AutoTokenizer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm

# Load your trained model/tokenizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = "Sujal-mT5"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = MT5ForConditionalGeneration.from_pretrained(model_path).to(device)
model.eval()

# Sentence similarity model
sim_model = SentenceTransformer("l3cube-pune/indic-sentence-similarity-sbert").to(device)

# BLEU smoother
smoother = SmoothingFunction().method2

# Languages and codes
langs = ['Kannada', 'Tamil', 'Telugu', 'Malayalam']
lang_code_map = {
    'Kannada': 'Kannada',
    'Tamil': 'Tamil',
    'Telugu': 'Telugu',
    'Malayalam': 'Malayalam'
}

# Dummy dataset (replace with your real dataset)
# Ensure df is already loaded and has columns for each language
# Example:
# df = pd.read_csv("your_multilingual_data.csv") or defined manually
df = pd.DataFrame({
    'Kannada': ['ನಾನು ಶಾಲೆಗೆ ಹೋಗುತ್ತಿದ್ದೇನೆ', 'ಅವನ ಹೆಸರು ರಾಮ'],
    'Tamil': ['நான் பள்ளிக்கு செல்கிறேன்', 'அவன் பெயர் ராம்'],
    'Telugu': ['నేను పాఠశాలకు వెళుతున్నాను', 'అతని పేరు రామ్'],
    'Malayalam': ['ഞാൻ സ്കൂളിലേക്ക് പോകുന്നു', 'അവന്റെ പേര് രാമൻ']
})

# Translation function
def translate_batch(texts, src_lang, tgt_lang):
    prefix = f"translate {lang_code_map[src_lang]} to {lang_code_map[tgt_lang]}: "
    inputs = tokenizer([prefix + text for text in texts], return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=128)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

# BLEU score
def sentence_bleu_score(reference, candidate):
    return sentence_bleu(
        [reference.split()],
        candidate.split(),
        weights=(0.25, 0.25, 0.25, 0.25),
        smoothing_function=smoother
    )

# Cosine similarity
def sentence_similarity(ref, cand):
    emb_ref = sim_model.encode(ref, convert_to_tensor=True)
    emb_cand = sim_model.encode(cand, convert_to_tensor=True)
    return util.pytorch_cos_sim(emb_ref, emb_cand).item()

# Initialize score matrices
bleu_df = pd.DataFrame(index=langs, columns=langs, dtype=float)
sim_df = pd.DataFrame(index=langs, columns=langs, dtype=float)

# Evaluation
for src_lang in langs:
    src_texts = df[src_lang].tolist()
    for tgt_lang in langs:
        tgt_texts = df[tgt_lang].tolist()

        if src_lang == tgt_lang:
            bleu_df.loc[src_lang, tgt_lang] = 100.0
            sim_df.loc[src_lang, tgt_lang] = 1.0
            print(f"Skipping {src_lang} -> {tgt_lang} (same language), BLEU=100, Similarity=1.0")
            continue

        print(f"\nTranslating {src_lang} -> {tgt_lang} ...")
        translations = translate_batch(src_texts, src_lang, tgt_lang)

        bleu_scores = []
        sim_scores = []

        for i, (ref, pred) in enumerate(tqdm(zip(tgt_texts, translations), total=len(tgt_texts), desc=f"{src_lang}->{tgt_lang}")):
            try:
                bleu_val = sentence_bleu_score(ref, pred)
            except Exception:
                bleu_val = 0.0
            sim_val = sentence_similarity(ref, pred)

            bleu_scores.append(bleu_val)
            sim_scores.append(sim_val)

        avg_bleu = np.mean(bleu_scores) * 100
        avg_sim = np.mean(sim_scores)

        bleu_df.loc[src_lang, tgt_lang] = avg_bleu
        sim_df.loc[src_lang, tgt_lang] = avg_sim

# Output matrices
print("\nBLEU Scores Matrix (%):")
print(bleu_df.round(2))

print("\nSimilarity Scores Matrix:")
print(sim_df.round(4))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Skipping Kannada -> Kannada (same language), BLEU=100, Similarity=1.0

Translating Kannada -> Tamil ...


Kannada->Tamil: 100%|██████████| 2/2 [00:00<00:00, 51.54it/s]



Translating Kannada -> Telugu ...


Kannada->Telugu: 100%|██████████| 2/2 [00:00<00:00, 52.79it/s]


...


Translating Malayalam -> Tamil ...


Malayalam->Tamil: 100%|██████████| 2/2 [00:00<00:00, 52.63it/s]



Translating Malayalam -> Telugu ...


Malayalam->Telugu: 100%|██████████| 2/2 [00:00<00:00, 52.65it/s]

Skipping Malayalam -> Malayalam (same language), BLEU=100, Similarity=1.0

BLEU Scores Matrix (%):
           Kannada   Tamil  Telugu  Malayalam
Kannada     100.00   44.84   15.97        0.0
Tamil        35.58  100.00   28.87        0.0
Telugu       40.88   15.97  100.00        0.0
Malayalam    49.28   15.97    0.00      100.0

Similarity Scores Matrix:
           Kannada   Tamil  Telugu  Malayalam
Kannada     1.0000  0.6484  0.5947     0.4021
Tamil       0.6787  1.0000  0.6695     0.3574
Telugu      0.6311  0.7788  1.0000     0.7056
Malayalam   0.6930  0.6731  0.5437     1.0000


In [22]:
import pandas as pd
import numpy as np
from transformers import MT5ForConditionalGeneration, AutoTokenizer
import torch
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load your trained model/tokenizer
model_path = "Sujal-mT5"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = MT5ForConditionalGeneration.from_pretrained(model_path).to(device)
model.eval()

# Load sentence similarity model
sim_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# BLEU smoother
smoother = SmoothingFunction().method4

langs = ['Kannada', 'Tamil', 'Telugu', 'Malayalam']
lang_code_map = {
    'Kannada': 'Kannada',
    'Tamil': 'Tamil',
    'Telugu': 'Telugu',
    'Malayalam': 'Malayalam'
}

# Load your parallel dataframe (replace with actual path)
df = pd.read_csv("final_dataset.csv") 
df=df.head(50) # Replace with your CSV file path

def translate_batch(texts, src_lang, tgt_lang):
    prefix = f"translate {lang_code_map[src_lang]} to {lang_code_map[tgt_lang]}: "
    inputs = tokenizer([prefix + text for text in texts], return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=128)
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return decoded

def sentence_bleu_score(reference, candidate):
    return sentence_bleu(
        [reference.split()],
        candidate.split(),
        weights=(0.25, 0.25, 0.25, 0.25),
        smoothing_function=smoother
    )

def sentence_similarity(ref, cand):
    emb_ref = sim_model.encode(ref, convert_to_tensor=True)
    emb_cand = sim_model.encode(cand, convert_to_tensor=True)
    return util.pytorch_cos_sim(emb_ref, emb_cand).item()

# Matrices for results
bleu_df = pd.DataFrame(index=langs, columns=langs, dtype=float)
sim_df = pd.DataFrame(index=langs, columns=langs, dtype=float)

# Compute BLEU and similarity
for src_lang in langs:
    src_texts = df[src_lang].tolist()
    for tgt_lang in langs:
        tgt_texts = df[tgt_lang].tolist()
        if src_lang == tgt_lang:
            bleu_df.loc[src_lang, tgt_lang] = 100.0
            sim_df.loc[src_lang, tgt_lang] = 1.0
            print(f"Skipping {src_lang} -> {tgt_lang} (same language), BLEU=100, Similarity=1.0")
            continue

        print(f"\nTranslating {src_lang} -> {tgt_lang} ...")
        translations = translate_batch(src_texts, src_lang, tgt_lang)

        bleu_scores = []
        sim_scores = []
        for i, (ref, pred) in enumerate(tqdm(zip(tgt_texts, translations), total=len(tgt_texts), desc=f"{src_lang}->{tgt_lang}")):
            try:
                bleu_val = sentence_bleu_score(ref, pred)
            except Exception:
                bleu_val = 0.0
            sim_val = sentence_similarity(ref, pred)
            bleu_scores.append(bleu_val)
            sim_scores.append(sim_val)

        avg_bleu = np.mean(bleu_scores) * 100
        avg_sim = np.mean(sim_scores)

        bleu_df.loc[src_lang, tgt_lang] = avg_bleu
        sim_df.loc[src_lang, tgt_lang] = avg_sim

print("\nBLEU Scores Matrix (%):")
print(bleu_df.round(2))

print("\nSimilarity Scores Matrix:")
print(sim_df.round(4))

# ==== Calculate Averages (excluding same-language) ====
mask = ~np.eye(len(langs), dtype=bool)
bleu_array = bleu_df.values.astype(float)
sim_array = sim_df.values.astype(float)

avg_bleu_score = bleu_array[mask].mean()
avg_sim_score = sim_array[mask].mean()

print(f"\nAverage BLEU Score (excluding same-language): {avg_bleu_score:.2f}%")
print(f"Average Cosine Similarity Score (excluding same-language): {avg_sim_score:.4f}")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Skipping Kannada -> Kannada (same language), BLEU=100, Similarity=1.0

Translating Kannada -> Tamil ...


Kannada->Tamil: 100%|██████████| 50/50 [00:00<00:00, 50.85it/s]



Translating Kannada -> Telugu ...


Kannada->Telugu: 100%|██████████| 50/50 [00:00<00:00, 51.36it/s]


...


Translating Malayalam -> Tamil ...


Malayalam->Tamil: 100%|██████████| 50/50 [00:01<00:00, 47.31it/s]



Translating Malayalam -> Telugu ...


Malayalam->Telugu: 100%|██████████| 50/50 [00:00<00:00, 50.91it/s]

Skipping Malayalam -> Malayalam (same language), BLEU=100, Similarity=1.0

BLEU Scores Matrix (%):
           Kannada   Tamil  Telugu  Malayalam
Kannada     100.00    2.21    1.54       1.37
Tamil         1.84  100.00    1.46       0.63
Telugu        1.45    1.21  100.00       0.51
Malayalam     1.71    1.17    1.42     100.00

Similarity Scores Matrix:
           Kannada   Tamil  Telugu  Malayalam
Kannada     1.0000  0.7304  0.7082     0.7265
Tamil       0.7796  1.0000  0.6925     0.7120
Telugu      0.7636  0.7276  1.0000     0.7320
Malayalam   0.7640  0.7243  0.7323     1.0000

Average BLEU Score (excluding same-language): 1.38%
Average Cosine Similarity Score (excluding same-language): 0.7327


In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import MT5ForConditionalGeneration, AutoTokenizer
from tqdm import tqdm
import sacrebleu

# Load model and tokenizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = "Sujal-mT5"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = MT5ForConditionalGeneration.from_pretrained(model_path).to(device)
model.eval()

# Dravidian language mapping
langs = ['Kannada', 'Tamil', 'Telugu', 'Malayalam']
lang_code_map = {
    'Kannada': 'Kannada',
    'Tamil': 'Tamil',
    'Telugu': 'Telugu',
    'Malayalam': 'Malayalam'
}

# 10-example dataset (you can replace this with your real one)
df = pd.DataFrame({
    'Kannada': [
        'ನಾನು ಶಾಲೆಗೆ ಹೋಗುತ್ತಿದ್ದೇನೆ', 'ಅವನ ಹೆಸರು ರಾಮ', 'ಅವಳು ಪುಸ್ತಕ ಓದುತ್ತಿದ್ದಾಳೆ',
        'ನಾವು ಮನೆಗೆ ಹೋಗುತ್ತೇವೆ', 'ಅವರು ಚೆನ್ನಾಗಿದ್ದಾರೆ', 'ನೀವು ಏನು ಮಾಡುತ್ತಿದ್ದೀರಿ?',
        'ಅವರು ಆಟವಾಡುತ್ತಿದ್ದಾರೆ', 'ಅದು ನನ್ನ ಕಾರು', 'ನಾನು ಭತ್ತ ತಿನ್ನುತ್ತಿದ್ದೇನೆ', 'ಅವನು ಸಿನಿಮಾ ನೋಡುತ್ತಿದ್ದಾನೆ'
    ],
    'Tamil': [
        'நான் பள்ளிக்கு செல்கிறேன்', 'அவன் பெயர் ராம்', 'அவள் புத்தகம் படிக்கிறாள்',
        'நாம் வீட்டுக்கு செல்கிறோம்', 'அவர்கள் நலமாக இருக்கிறார்கள்', 'நீங்கள் என்ன செய்து கொண்டிருக்கிறீர்கள்?',
        'அவர்கள் விளையாடுகிறார்கள்', 'அது என் கார்', 'நான் அரிசி சாப்பிடுகிறேன்', 'அவன் படம் பார்க்கிறான்'
    ],
    'Telugu': [
        'నేను పాఠశాలకు వెళుతున్నాను', 'అతని పేరు రామ్', 'ఆమె పుస్తకం చదువుతుంది',
        'మేము ఇంటికి వెళ్తున్నాము', 'వారు బాగున్నారు', 'మీరు ఏమి చేస్తున్నారు?',
        'వారు ఆడుతున్నారు', 'అది నా కారు', 'నేను అన్నం తింటున్నాను', 'అతను సినిమా చూస్తున్నాడు'
    ],
    'Malayalam': [
        'ഞാൻ സ്കൂളിലേക്ക് പോകുന്നു', 'അവന്റെ പേര് രാമൻ', 'അവൾ പുസ്തകം വായിക്കുന്നു',
        'ഞങ്ങൾ വീട്ടിലേക്ക് പോകുന്നു', 'അവർ സുഖമായിരിക്കുന്നു', 'നിങ്ങൾ എന്താണ് ചെയ്യുന്നത്?',
        'അവർ കളിക്കുന്നു', 'അത് എന്റെ കാർ ആണ്', 'ഞാൻ അരി കഴിക്കുന്നു', 'അവൻ സിനിമ കാണുന്നു'
    ]
})

# Translation function
def translate_batch(texts, src_lang, tgt_lang):
    prefix = f"translate {lang_code_map[src_lang]} to {lang_code_map[tgt_lang]}: "
    inputs = tokenizer([prefix + text for text in texts], return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=128)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

# chrF++ Score Calculation Function
def compute_chrfpp(refs, preds):
    refs = [[ref] for ref in refs]  # sacrebleu expects list of references per sentence
    chrf_score = sacrebleu.corpus_chrf(preds, refs, word_order=2)
    return chrf_score.score

# Score matrix
chrf_df = pd.DataFrame(index=langs, columns=langs, dtype=float)

# Evaluate translation directions
for src_lang in langs:
    src_texts = df[src_lang].tolist()
    for tgt_lang in langs:
        tgt_texts = df[tgt_lang].tolist()

        if src_lang == tgt_lang:
            chrf_df.loc[src_lang, tgt_lang] = 100.0
            print(f"Skipping {src_lang} -> {tgt_lang} (same language), chrF++ = 100.0")
            continue

        print(f"\nTranslating {src_lang} -> {tgt_lang} ...")
        translations = translate_batch(src_texts, src_lang, tgt_lang)

        # Compute chrF++
        chrf_score = compute_chrfpp(tgt_texts, translations)
        chrf_df.loc[src_lang, tgt_lang] = chrf_score

# Output results
print("\nchrF++ Score Matrix (%):")
print(chrf_df.round(2))


2025-05-15 11:14:26.619437: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-15 11:14:26.945063: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747307667.073740    1899 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747307667.108781    1899 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747307667.385536    1899 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Skipping Kannada -> Kannada (same language), chrF++ = 100.0

Translating Kannada -> Tamil ...

Translating Kannada -> Telugu ...

Translating Kannada -> Malayalam ...

Translating Tamil -> Kannada ...
Skipping Tamil -> Tamil (same language), chrF++ = 100.0

Translating Tamil -> Telugu ...

Translating Tamil -> Malayalam ...

Translating Telugu -> Kannada ...

Translating Telugu -> Tamil ...
Skipping Telugu -> Telugu (same language), chrF++ = 100.0

Translating Telugu -> Malayalam ...

Translating Malayalam -> Kannada ...

Translating Malayalam -> Tamil ...

Translating Malayalam -> Telugu ...
Skipping Malayalam -> Malayalam (same language), chrF++ = 100.0

chrF++ Score Matrix (%):
           Kannada   Tamil  Telugu  Malayalam
Kannada     100.00   76.99   17.67      13.41
Tamil        45.12  100.00   13.13      21.68
Telugu       67.08   34.54  100.00      16.50
Malayalam    69.09   34.99   20.10     100.00


In [2]:
import pandas as pd
import torch
from transformers import MT5ForConditionalGeneration, AutoTokenizer
from tqdm import tqdm
import sacrebleu

# Load model and tokenizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = "Sujal-mT5"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = MT5ForConditionalGeneration.from_pretrained(model_path).to(device)
model.eval()

# Language list and mapping
langs = ['Kannada', 'Tamil', 'Telugu', 'Malayalam']
lang_code_map = {
    'Kannada': 'Kannada',
    'Tamil': 'Tamil',
    'Telugu': 'Telugu',
    'Malayalam': 'Malayalam'
}

# Load your CSV dataset (make sure it has the columns: Kannada, Tamil, Telugu, Malayalam)
df = pd.read_csv("final_dataset.csv")
df=df.head(50)

# Limit to 10 rows to compute chrF++ (as requested)
df = df[langs].dropna().head(10)

# Translation function
def translate_batch(texts, src_lang, tgt_lang):
    prefix = f"translate {lang_code_map[src_lang]} to {lang_code_map[tgt_lang]}: "
    inputs = tokenizer([prefix + text for text in texts], return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=128)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

# chrF++ scoring
def compute_chrfpp(refs, preds):
    refs = [[ref] for ref in refs]  # sacrebleu expects list of references per sentence
    chrf_score = sacrebleu.corpus_chrf(preds, refs, word_order=2)
    return chrf_score.score

# chrF++ score matrix
chrf_df = pd.DataFrame(index=langs, columns=langs, dtype=float)

# Compute scores
for src_lang in langs:
    src_texts = df[src_lang].tolist()
    for tgt_lang in langs:
        tgt_texts = df[tgt_lang].tolist()

        if src_lang == tgt_lang:
            chrf_df.loc[src_lang, tgt_lang] = 100.0
            print(f"Skipping {src_lang} -> {tgt_lang} (same language), chrF++ = 100.0")
            continue

        print(f"\nTranslating {src_lang} -> {tgt_lang} ...")
        translations = translate_batch(src_texts, src_lang, tgt_lang)
        chrf_score = compute_chrfpp(tgt_texts, translations)
        chrf_df.loc[src_lang, tgt_lang] = chrf_score

# Print results
print("\nchrF++ Score Matrix (%):")
print(chrf_df.round(2))

# Optionally save
chrf_df.to_csv("chrf_scores_matrix.csv")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Skipping Kannada -> Kannada (same language), chrF++ = 100.0

Translating Kannada -> Tamil ...

Translating Kannada -> Telugu ...

Translating Kannada -> Malayalam ...

Translating Tamil -> Kannada ...
Skipping Tamil -> Tamil (same language), chrF++ = 100.0

Translating Tamil -> Telugu ...

Translating Tamil -> Malayalam ...

Translating Telugu -> Kannada ...

Translating Telugu -> Tamil ...
Skipping Telugu -> Telugu (same language), chrF++ = 100.0

Translating Telugu -> Malayalam ...

Translating Malayalam -> Kannada ...

Translating Malayalam -> Tamil ...

Translating Malayalam -> Telugu ...
Skipping Malayalam -> Malayalam (same language), chrF++ = 100.0

chrF++ Score Matrix (%):
           Kannada   Tamil  Telugu  Malayalam
Kannada     100.00   15.53   15.05      11.29
Tamil        40.43  100.00    9.39       7.47
Telugu       11.65    9.40  100.00      11.66
Malayalam    12.05    7.95    5.32     100.00
